In [16]:
import numpy as np
from nltk.tokenize import word_tokenize
from lstm_vae import create_lstm_vae, inference
import keras
import sys, time
from keras.callbacks import CSVLogger, Callback

from tensorflow import set_random_seed
set_random_seed(1234)
np.random.seed(1234)


def get_text_data(data_path, num_samples=1000):

    # vectorize the data
    input_texts = []
    input_characters = set(["\t"])

    with open(data_path, "r", encoding="utf-8") as f:
        #lines = f.read().lower().split('\n')
        lines = f.read().lower().split('\n')
        
    for line in lines[: min(num_samples, len(lines) - 1)]:

        #input_text, _= line.split('\t')
        input_text = line
        input_text = word_tokenize(input_text)
        input_text.append("<end>")

        input_texts.append(input_text)

        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)

    input_characters = sorted(list(input_characters))
    num_encoder_tokens = len(input_characters)
    max_encoder_seq_length = max([len(txt) for txt in input_texts]) + 1

    print("Number of samples:", len(input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)

    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())

    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    decoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

    for i, input_text in enumerate(input_texts):
        decoder_input_data[i, 0, input_token_index["\t"]] = 1.0

        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
            decoder_input_data[i, t + 1, input_token_index[char]] = 1.0

    return max_encoder_seq_length, num_encoder_tokens, input_characters, input_token_index, reverse_input_char_index, \
           encoder_input_data, decoder_input_data


def main(params):
    
    num_samples = int(params['num_samples'])
    data_path = "data/" + params['dataset']
    dataname = params['dataset'].split('.')[0]
    
    batch_size = int(params['batch_size'])
    latent_dim = int(params['latent_dim'])
    intermediate_dim = int(params['intermediate_dim'])
    epochs = int(params['epochs'])
    
    train = int(params['train'])
    save = int(params['save'])
    load = int(params['load'])
    
    timesteps_max, enc_tokens, characters, char2id, id2char, x, x_decoder = get_text_data(num_samples=num_samples,
                                                                                          data_path=data_path)

    print(x.shape, "Creating model...")
    
    input_dim = x.shape[-1]
    timesteps = x.shape[-2]
    
    if load:
        print("Loading model ... ")
        
        #vae = keras.models.load_model("models/vae_{}.h5".format(dataname))
        enc = keras.models.load_model("models/encoder_{}.h5".format(dataname))
        gen = keras.models.load_model("models/generator_{}.h5".format(dataname))
        stepper = keras.models.load_model("models/stepper_{}.h5".format(dataname))
    
    if train:
        print("Training model...")
        
        vae, enc, gen, stepper = create_lstm_vae(input_dim,
                                             batch_size=batch_size,
                                             intermediate_dim=intermediate_dim,
                                             latent_dim=latent_dim)

        
        csv_logger = CSVLogger('training_vae.log', separator=',', append=False)
        vae.fit([x, x_decoder], x, epochs=epochs, verbose=1, callbacks=[csv_logger])
        
        if save:
            print("Saving model ... ")
            
            vae.save("models/vae_{}.h5".format(dataname))
            enc.save("models/encoder_{}.h5".format(dataname))
            gen.save("models/generator_{}.h5".format(dataname))
            stepper.save("models/stepper_{}.h5".format(dataname))
    
    print("Fitted, predicting...")


    def decode(s, start_char = "\t"):
        return inference.decode_sequence(s, gen, stepper, input_dim, char2id, id2char, timesteps_max, start_char = start_char)

    def continue_seq(x_start):
        return inference.continue_sequence(x_start, gen, stepper, input_dim, char2id, id2char, timesteps_max)

    for _ in range(5):

        id_from = np.random.randint(0, x.shape[0] - 1)
        id_to = np.random.randint(0, x.shape[0] - 1)

        m_from, std_from = enc.predict([[x[id_from]]])
        m_to, std_to = enc.predict([[x[id_to]]])

        seq_from = np.random.normal(size=(latent_dim,))
        seq_from = m_from + std_from * seq_from

        seq_to = np.random.normal(size=(latent_dim,))
        seq_to = m_to + std_to * seq_to

        print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_from], axis=1)]), "==")

        for v in np.linspace(0, 1, 7):
            print("%.2f\t" % (1 - v), decode(v * seq_to + (1 - v) * seq_from))

        print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_to], axis=1)]), "==")
        
    for _ in range(20):
        id_sentence = np.random.randint(0, x.shape[0] - 1)
        
        n_words = np.sum(x[id_sentence])
        n_kept = np.random.randint(n_words//2, n_words-1)
        
        new_x = np.zeros((x[id_sentence].shape))
        new_x[:n_kept,:] = x[id_sentence,:n_kept,:]
        
        print("==  \t", " ".join([id2char[j] for j in np.argmax(new_x[:n_kept], axis=1)]), " ... \t\t ==")
        
        print("\t...\t", continue_seq(new_x))
            
        print("==  \t", " ".join([id2char[j] for j in np.argmax(x[id_sentence], axis=1)]), "==")
        

In [9]:
data_path = './data/europ.en' 
with open(data_path, "r", encoding="utf-8") as f:
        #lines = f.read().lower().split('\n')
        lines = f.read().lower().split('\n')

In [12]:
lines[2]

"although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful."

In [18]:
config_file_name = 'config.txt'

with open(config_file_name, 'r') as f:
    lines = f.readlines()

params = {}

for line in lines:
    line = line.split('\n')[0]
    param_list = line.split(' ')
    param_name = param_list[0]
    param_value = param_list[1]
    params[param_name] = param_value

main(params)

Number of samples: 100
Number of unique input tokens: 750
Max sequence length for inputs: 117
(100, 117, 750) Creating model...
Training model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, None, 750)    0                                            
__________________________________________________________________________________________________
lstm_9 (LSTM)                   (None, 353)          1558848     input_21[0][0]                   
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 191)          67614       lstm_9[0][0]                     
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 191)          67614    

100/100 [==============================] - 11s 109ms/step - loss: 1.2574 - xent_loss: 1.2562 - kl_loss: 0.0012
Epoch 46/200
100/100 [==============================] - 11s 110ms/step - loss: 1.2545 - xent_loss: 1.2534 - kl_loss: 0.0012
Epoch 47/200
100/100 [==============================] - 11s 110ms/step - loss: 1.2532 - xent_loss: 1.2519 - kl_loss: 0.0013
Epoch 48/200
100/100 [==============================] - 11s 112ms/step - loss: 1.2491 - xent_loss: 1.2480 - kl_loss: 0.0011
Epoch 49/200
100/100 [==============================] - 11s 114ms/step - loss: 1.2451 - xent_loss: 1.2439 - kl_loss: 0.0012
Epoch 50/200
100/100 [==============================] - 11s 112ms/step - loss: 1.2411 - xent_loss: 1.2399 - kl_loss: 0.0011
Epoch 51/200
100/100 [==============================] - 11s 114ms/step - loss: 1.2391 - xent_loss: 1.2379 - kl_loss: 0.0012
Epoch 52/200
100/100 [==============================] - 11s 112ms/step - loss: 1.2368 - xent_loss: 1.2358 - kl_loss: 0.0010
Epoch 53/200
100/100 

100/100 [==============================] - 11s 113ms/step - loss: 0.9492 - xent_loss: 0.9485 - kl_loss: 6.9643e-04
Epoch 110/200
100/100 [==============================] - 11s 113ms/step - loss: 0.9410 - xent_loss: 0.9403 - kl_loss: 6.9243e-04
Epoch 111/200
100/100 [==============================] - 12s 117ms/step - loss: 0.9362 - xent_loss: 0.9355 - kl_loss: 6.3446e-04
Epoch 112/200
100/100 [==============================] - 12s 118ms/step - loss: 0.9284 - xent_loss: 0.9278 - kl_loss: 6.1698e-04
Epoch 113/200
100/100 [==============================] - 11s 110ms/step - loss: 0.9229 - xent_loss: 0.9223 - kl_loss: 6.2052e-04
Epoch 114/200
100/100 [==============================] - 11s 107ms/step - loss: 0.9171 - xent_loss: 0.9165 - kl_loss: 6.0840e-04
Epoch 115/200
100/100 [==============================] - 11s 110ms/step - loss: 0.9101 - xent_loss: 0.9095 - kl_loss: 6.1723e-04
Epoch 116/200
100/100 [==============================] - 11s 106ms/step - loss: 0.9076 - xent_loss: 0.9069 - kl

100/100 [==============================] - 10s 99ms/step - loss: 0.5369 - xent_loss: 0.5364 - kl_loss: 5.5639e-04
Epoch 173/200
100/100 [==============================] - 12s 119ms/step - loss: 0.5347 - xent_loss: 0.5341 - kl_loss: 5.6583e-04
Epoch 174/200
100/100 [==============================] - 11s 106ms/step - loss: 0.5356 - xent_loss: 0.5350 - kl_loss: 5.7903e-04
Epoch 175/200
100/100 [==============================] - 11s 106ms/step - loss: 0.5288 - xent_loss: 0.5283 - kl_loss: 5.6981e-04
Epoch 176/200
100/100 [==============================] - 11s 113ms/step - loss: 0.5197 - xent_loss: 0.5191 - kl_loss: 5.4943e-04
Epoch 177/200
100/100 [==============================] - 11s 115ms/step - loss: 0.5122 - xent_loss: 0.5117 - kl_loss: 5.1559e-04
Epoch 178/200
100/100 [==============================] - 11s 114ms/step - loss: 0.5070 - xent_loss: 0.5065 - kl_loss: 4.8892e-04
Epoch 179/200
100/100 [==============================] - 11s 106ms/step - loss: 0.5020 - xent_loss: 0.5015 - kl_

/home/axeldelavergne/anaconda3/envs/first_env/lib/python3.5/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_10 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'dense_19/BiasAdd:0' shape=(?, 353) dtype=float32>, <tf.Tensor 'dense_19/BiasAdd:0' shape=(?, 353) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/axeldelavergne/anaconda3/envs/first_env/lib/python3.5/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_10 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'dense_19_1/BiasAdd:0' shape=(?, 353) dtype=float32>, <tf.Tensor 'dense_19_1/BiasAdd:0' shape=(?, 353) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/axeldelavergne/anaconda3/envs/first_env/lib/python3.5/site-packages/kera

Fitted, predicting...
==  	 but , madam president , my personal request has not been met . <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
1.00	 i would like to advice about rule 143 concerning inadmissibility . <end> 
0.83	 i would like to advice about rule 143 concerning inadmissibility . <end> 
0.67	 i would like to advice about rule 143 concerning inadmissibility . <end> 
0.50	 i would like to advice about rule 143 concerning inadmissibility . <end> 
0.33	 i would like to advice about rule 143 concerning inadmissibility . <end> 
0.17	 i would like to advice about rule 143 concerning inadmissibility . <end> 
0.00	 i would like to advice about rule 143 concerning inadmissibility . <end> 
==  	 the cunha report on multiannual guidance programmes comes before parliament on thursday and contains a proposal in paragraph 6 t

	...	 . <end> 
==  	 given that the commission is represented by vice-president de palacio , i believe that , before voting , it would help if the commission could let us know how ready it is to present this programme , as agreed . alternatively , parliament is not ready to examine this programme , as some appear to be suggesting . <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
==  	 but , madam president , my personal request has not been met  ... 		 ==
	...	 . <end> 
==  	 but , madam president , my personal request has not been met . <end> 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 	 ==
==  	 therefore , madam president , i would ask you to request that the commission express its opinion  ... 		 ==
	...	 , we had agreed - , the commission is not a 